In [ ]:
import csv
import pandas as pd
import sys
import time
import numpy as np
# pd.set_option('display.max_colwidth', None)
# pd.set_option('max_columns', None)
# sys.path.append('../')
# from utils.bigquery import BigqueryClient
# project_id = "shpsec-ssdp-prod"
# service_account_file = "../secrets/shpsec-mfd-dv-bigquery.json"
# bq_client = BigqueryClient(project_id, service_account_file)

In [ ]:
# Dataset struction
'''
company name
a 0
b 1
c 2
d  3
e 4
f 5
g 6
h 7
i 8
j 9
k 10
l 11
m 12
n 13

folder name 
screenFeature 0
pluginFeature 1
dateFeature 2
browserFeature 3
canvasFeature 4
fontsFeature 5
browser + os  6
verndor+renderer 7
6 + 7     8
city+region     9
devicePixelRatio 10
screem(same with 0) 11

The path:
bfp/ + featurename + companyname + A(attack)/N(normal)
In each file .csv
raw_data rate(of the whole number)

'''


In [ ]:
# attack& normal  A&N

In [ ]:
# This is a demo to analyze the company 0
compare_data = []
num = 12
for i in range(num):
    compare_data.append([])
    attck_file = open('bfp/' + str(i)+ '/0A.csv','r',newline='')
    normal_file = open('bfp/' + str(i)+ '/0N.csv','r',newline='')
    attack_reader_lines = csv.reader(attck_file)
    normal_reader_lines = csv.reader(normal_file)
    normal_data = {}
    for raw_data in normal_reader_lines:
        normal_data[raw_data[0]] = raw_data[1]
    for raw_data in attack_reader_lines:
        #min_val is a demo value, it just remove some noise
        min_val = 0.001
        if float(raw_data[1]) < 0.001:
            break
        if raw_data[0] in normal_data:
            compare_data[i].append([raw_data[0], raw_data[1], normal_data[raw_data[0]]])
        else:
            compare_data[i].append([raw_data[0], raw_data[1], 0])
            
for i in range(num):
    print(compare_data[i])
    print("=============================")


In [ ]:
#analyze the possible data in the dataset
for i in range(num):
    sign = []
    for j in range(len(compare_data[i])):
        if (float(compare_data[i][j][1]) / 5 > float(compare_data[i][j][2])):
            sign.append(compare_data[i][j][0])
    print(sign)
    print('=============================')

In [ ]:
# find same database
# The main part is compare attack & normal data

In [ ]:
num_company = 14
num = 12
# This is the demo company list name
company = ['a','b','c','d','e','f','g','h','i','j','k','l','m','n']
feature = ['screenFeature','pluginFeature','dateFeature','browserFeature','canvasFeature','fontsFeature','browser+os','verndor+renderer','browser+os+verndor+renderer ','city+region','devicePixelRatio','screensize']
print(feature)
sign = [];
for i in range(num):
    sign.append({})
for j in range(num_company):
    compare_data = []
    for i in range(num):
        compare_data.append([])
        attck_file = open('bfp/' + str(i)+ '/' + str(j) +'A.csv','r',newline='', encoding="utf-8")
        normal_file = open('bfp/' + str(i)+ '/'+ str(j) +'N.csv','r',newline='', encoding="utf-8")
        attack_reader_lines = csv.reader(x.replace('\0', '') for x in attck_file)
        normal_reader_lines = csv.reader(x.replace('\0', '') for x in normal_file)
        normal_data = {}
        for raw_data in normal_reader_lines:
            normal_data[raw_data[0]] = raw_data[1]
        for raw_data in attack_reader_lines:
            if float(raw_data[1]) < 0.0005:
                break
            if raw_data[0] in normal_data:
                if (float(raw_data[1])/5 > float(normal_data[raw_data[0]])):
                    compare_data[i].append([raw_data[0], raw_data[1], normal_data[raw_data[0]]])
            else:
                compare_data[i].append([raw_data[0], raw_data[1], 0])
    sign_len = []
    for i in range(num):
        sign_len.append(len(compare_data[i]))
    print(company[j],":",sign_len)
    for i in range(num):
        for element in compare_data[i]:
            if element[0] in sign[i]:
                sign[i][element[0]] += 1
            else:
                sign[i][element[0]] = 1

    
        


In [ ]:
# distinect number // unique

In [ ]:
num_company = 14
num = 12
company = ['a','b','c','d','e','f','g','h','i','j','k','l','m','n']
feature = ['screenFeature','pluginFeature','dateFeature','browserFeature','canvasFeature','fontsFeature','browser+os','verndor+renderer','browser+os+verndor+renderer ','city+region','devicePixelRatio','screensize']
for i in range(num):
    attack_dic = {}
    nor_dic = {}
    for j in range(num_company):
        attck_file = open('bfp/' + str(i)+ '/' + str(j) +'A.csv','r',newline='', encoding="utf-8")
        normal_file = open('bfp/' + str(i)+ '/'+ str(j) +'N.csv','r',newline='', encoding="utf-8")
        attack_reader_lines = csv.reader(x.replace('\0', '') for x in attck_file)
        normal_reader_lines = csv.reader(x.replace('\0', '') for x in normal_file)
        for raw_data in normal_reader_lines:
            nor_dic[raw_data[0]] = raw_data[1]
        for raw_data in attack_reader_lines:
            attack_dic[raw_data[0]] = raw_data[1]
    print(feature[i],'=================')
    unique_normal = 0
    for ele in nor_dic:
        if ele not in attack_dic:
            unique_normal += 1
    print('normal',len(nor_dic),unique_normal/len(nor_dic))
    unique_attack = 0
    for ele in attack_dic:
        if ele not in nor_dic:
            unique_attack += 1
    print('attack',len(attack_dic),unique_attack/len(attack_dic))
        


In [ ]:
# print(sign)
for i in range(num):
    a = dict(sorted(sign[i].items(), key=lambda x: x[-1], reverse=True))
    b = {}
    arr = []
    count = 0
    for element in a:
        if a[element] > 1:
            b[element] = a[element]
            count += 1
        if element not in {'MISSING','','nan','NULL'} and a[element] != 1:
            arr.append(a[element])
    
    nparr = np.array(arr)
    
    
    print("\n")
    print(feature[i],i,":================================")
    print("number:",count,"=====================================\n")
    print(len(nparr))
    print(np.mean(nparr), np.std(nparr))
    print(b)

In [ ]:
a = dict(sorted(sign[9].items(), key=lambda x: x[-1], reverse=True))
b = {}
arr = []
count = 0
for element in a:
    if a[element] > 1:
        b[element[]] = a[element]
        count += 1
    if element not in {'MISSING','','nan','NULL'} and a[element] != 1:
        arr.append(a[element])
print(b)

In [ ]:
#browser+os
num_company = 14
num = 12
company = ['a','b','c','d','e','f','g','h','i','j','k','l','m','n']
feature = ['screenFeature','pluginFeature','dateFeature','browserFeature','canvasFeature','fontsFeature','browser+os','verndor+renderer','browser+os+verndor+renderer ','city+region','devicePixelRatio','screensize']
# print(feature)
Normal_sign = {};
Attack_sign = {};
# for i in range(num_company):
#     Normal_sign.append({})
#     Attack_sign.append({})
# print(sign)
for j in range(num_company):
    compare_data = []
    i = 6
    compare_data.append([])
    attck_file = open('bfp/' + str(i)+ '/' + str(j) +'A.csv','r',newline='')
    normal_file = open('bfp/' + str(i)+ '/'+ str(j) +'N.csv','r',newline='')
    attack_reader_lines = csv.reader(x.replace('\0', '') for x in attck_file)
    normal_reader_lines = csv.reader(x.replace('\0', '') for x in normal_file)
    normal_data = {}
    attack_data = {}
    for raw_data in normal_reader_lines:
        if float(raw_data[1]) > 0.01:
            normal_data[raw_data[0]] = raw_data[1]
            if raw_data[0] in Normal_sign:
                Normal_sign[raw_data[0]] = [Normal_sign[raw_data[0]][0] + 1, Normal_sign[raw_data[0]][1] + float(raw_data[1])]
            else:
                Normal_sign[raw_data[0]] = [1, float(raw_data[1])]
    for raw_data in attack_reader_lines:
        if float(raw_data[1]) > 0.01:
            attack_data[raw_data[0]] = raw_data[1]
            if raw_data[0] in Attack_sign:
                Attack_sign[raw_data[0]] = [Attack_sign[raw_data[0]][0] + 1, Attack_sign[raw_data[0]][1] + float(raw_data[1])]
            else:
                Attack_sign[raw_data[0]] = [1, float(raw_data[1])]
    print("\n")
    print(company[j],":================================")
    a = dict(sorted(normal_data.items(), key=lambda x: x[-1], reverse=True))
    b = dict(sorted(attack_data.items(), key=lambda x: x[-1], reverse=True))
    print(a)
    print(b)
print('!!!!!!!!!!!!!!!!!!!!!!')
for key in Normal_sign:
    Normal_sign[key][1] = float(Normal_sign[key][1]) / float(Normal_sign[key][0])
for key in Attack_sign:
    Attack_sign[key][1] = float(Attack_sign[key][1]) / float(Attack_sign[key][0])
a = dict(sorted(Normal_sign.items(), key=lambda x: x[-1], reverse=True))
b = dict(sorted(Attack_sign.items(), key=lambda x: x[-1], reverse=True))
print(Normal_sign)
print(Attack_sign)

In [ ]:
#verndor+renderer
num_company = 14
num = 12
company = ['a','b','c','d','e','f','g','h','i','j','k','l','m','n']
feature = ['screenFeature','pluginFeature','dateFeature','browserFeature','canvasFeature','fontsFeature','browser+os','verndor+renderer','browser+os+verndor+renderer ','city+region','devicePixelRatio','screensize']
# print(feature)
Normal_sign = {};
Attack_sign = {};
# for i in range(num_company):
#     Normal_sign.append({})
#     Attack_sign.append({})
# print(sign)
for j in range(num_company):
    compare_data = []
    i = 7
    compare_data.append([])
    attck_file = open('bfp/' + str(i)+ '/' + str(j) +'A.csv','r',newline='')
    normal_file = open('bfp/' + str(i)+ '/'+ str(j) +'N.csv','r',newline='')
    attack_reader_lines = csv.reader(x.replace('\0', '') for x in attck_file)
    normal_reader_lines = csv.reader(x.replace('\0', '') for x in normal_file)
    normal_data = {}
    attack_data = {}
    for raw_data in normal_reader_lines:
        if float(raw_data[1]) > 0.01:
            normal_data[raw_data[0]] = raw_data[1]
            if raw_data[0] in Normal_sign:
                Normal_sign[raw_data[0]] = [Normal_sign[raw_data[0]][0] + 1, Normal_sign[raw_data[0]][1] + float(raw_data[1])]
            else:
                Normal_sign[raw_data[0]] = [1, float(raw_data[1])]
    for raw_data in attack_reader_lines:
        if float(raw_data[1]) > 0.01:
            attack_data[raw_data[0]] = raw_data[1]
            if raw_data[0] in Attack_sign:
                Attack_sign[raw_data[0]] = [Attack_sign[raw_data[0]][0] + 1, Attack_sign[raw_data[0]][1] + float(raw_data[1])]
            else:
                Attack_sign[raw_data[0]] = [1, float(raw_data[1])]
    print("\n")
    print(company[j],":================================")
    a = dict(sorted(normal_data.items(), key=lambda x: x[-1], reverse=True))
    b = dict(sorted(attack_data.items(), key=lambda x: x[-1], reverse=True))
    print(a)
    print(b)
print('!!!!!!!!!!!!!!!!!!!!!!')
for key in Normal_sign:
    Normal_sign[key][1] = float(Normal_sign[key][1]) / float(Normal_sign[key][0])
for key in Attack_sign:
    Attack_sign[key][1] = float(Attack_sign[key][1]) / float(Attack_sign[key][0])
a = dict(sorted(Normal_sign.items(), key=lambda x: x[-1], reverse=True))
b = dict(sorted(Attack_sign.items(), key=lambda x: x[-1], reverse=True))
print(Normal_sign)
print(Attack_sign)

In [ ]:
#browser+os+verndor+renderer
num_company = 14
num = 12
company = ['a','b','c','d','e','f','g','h','i','j','k','l','m','n']
feature = ['screenFeature','pluginFeature','dateFeature','browserFeature','canvasFeature','fontsFeature','browser+os','verndor+renderer','browser+os+verndor+renderer ','city+region','devicePixelRatio','screensize']
# print(feature)
Normal_sign = {};
Attack_sign = {};
# for i in range(num_company):
#     Normal_sign.append({})
#     Attack_sign.append({})
# print(sign)
for j in range(num_company):
    compare_data = []
    i = 8
    compare_data.append([])
    attck_file = open('bfp/' + str(i)+ '/' + str(j) +'A.csv','r',newline='')
    normal_file = open('bfp/' + str(i)+ '/'+ str(j) +'N.csv','r',newline='')
    attack_reader_lines = csv.reader(x.replace('\0', '') for x in attck_file)
    normal_reader_lines = csv.reader(x.replace('\0', '') for x in normal_file)
    normal_data = {}
    attack_data = {}
    for raw_data in normal_reader_lines:
        if float(raw_data[1]) > 0.01:
            normal_data[raw_data[0]] = raw_data[1]
            if raw_data[0] in Normal_sign:
                Normal_sign[raw_data[0]] = [Normal_sign[raw_data[0]][0] + 1, Normal_sign[raw_data[0]][1] + float(raw_data[1])]
            else:
                Normal_sign[raw_data[0]] = [1, float(raw_data[1])]
    for raw_data in attack_reader_lines:
        if float(raw_data[1]) > 0.01:
            attack_data[raw_data[0]] = raw_data[1]
            if raw_data[0] in Attack_sign:
                Attack_sign[raw_data[0]] = [Attack_sign[raw_data[0]][0] + 1, Attack_sign[raw_data[0]][1] + float(raw_data[1])]
            else:
                Attack_sign[raw_data[0]] = [1, float(raw_data[1])]
    print("\n")
    print(company[j],":================================")
    a = dict(sorted(normal_data.items(), key=lambda x: x[-1], reverse=True))
    b = dict(sorted(attack_data.items(), key=lambda x: x[-1], reverse=True))
    print(a)
    print(b)
print('!!!!!!!!!!!!!!!!!!!!!!')
for key in Normal_sign:
    Normal_sign[key][1] = float(Normal_sign[key][1]) / float(Normal_sign[key][0])
for key in Attack_sign:
    Attack_sign[key][1] = float(Attack_sign[key][1]) / float(Attack_sign[key][0])
a = dict(sorted(Normal_sign.items(), key=lambda x: x[-1], reverse=True))
b = dict(sorted(Attack_sign.items(), key=lambda x: x[-1], reverse=True))
print(Normal_sign)
print(Attack_sign)

In [ ]:
#city+region
num_company = 14
num = 12
company = ['a','b','c','d','e','f','g','h','i','j','k','l','m','n']
feature = ['screenFeature','pluginFeature','dateFeature','browserFeature','canvasFeature','fontsFeature','browser+os','verndor+renderer','browser+os+verndor+renderer ','city+region','devicePixelRatio','screensize']
# print(feature)
Normal_sign = {};
Attack_sign = {};
# for i in range(num_company):
#     Normal_sign.append({})
#     Attack_sign.append({})
# print(sign)
for j in range(num_company):
    compare_data = []
    i = 9
    compare_data.append([])
    attck_file = open('bfp/' + str(i)+ '/' + str(j) +'A.csv','r',newline='', encoding="utf-8")
    normal_file = open('bfp/' + str(i)+ '/'+ str(j) +'N.csv','r',newline='', encoding="utf-8")
    attack_reader_lines = csv.reader(x.replace('\0', '') for x in attck_file)
    normal_reader_lines = csv.reader(x.replace('\0', '') for x in normal_file)
    normal_data = {}
    attack_data = {}
    for raw_data in normal_reader_lines:
        if float(raw_data[1]) > 0.01:
            normal_data[raw_data[0]] = raw_data[1]
            if raw_data[0] in Normal_sign:
                Normal_sign[raw_data[0]] = [Normal_sign[raw_data[0]][0] + 1, Normal_sign[raw_data[0]][1] + float(raw_data[1])]
            else:
                Normal_sign[raw_data[0]] = [1, float(raw_data[1])]
    for raw_data in attack_reader_lines:
        if float(raw_data[1]) > 0.01:
            attack_data[raw_data[0]] = raw_data[1]
            if raw_data[0] in Attack_sign:
                Attack_sign[raw_data[0]] = [Attack_sign[raw_data[0]][0] + 1, Attack_sign[raw_data[0]][1] + float(raw_data[1])]
            else:
                Attack_sign[raw_data[0]] = [1, float(raw_data[1])]
    print("\n")
    print(company[j],":================================")
    a = dict(sorted(normal_data.items(), key=lambda x: x[-1], reverse=True))
    b = dict(sorted(attack_data.items(), key=lambda x: x[-1], reverse=True))
    print(a)
    print(b)
print('!!!!!!!!!!!!!!!!!!!!!!')
for key in Normal_sign:
    Normal_sign[key][1] = float(Normal_sign[key][1]) / float(Normal_sign[key][0])
for key in Attack_sign:
    Attack_sign[key][1] = float(Attack_sign[key][1]) / float(Attack_sign[key][0])
a = dict(sorted(Normal_sign.items(), key=lambda x: x[-1], reverse=True))
b = dict(sorted(Attack_sign.items(), key=lambda x: x[-1], reverse=True))
print(Normal_sign)
print(Attack_sign)

In [ ]:
# print(Normal_sign)
print(sorted(Normal_sign.items(), key = lambda i:i[1][0],reverse = -1) )

In [ ]:
print(sorted(Attack_sign.items(), key = lambda i:i[1][0],reverse = -1) )

In [ ]:
new_attack = {}
for ele in Attack_sign:
    name = ele[:2]
#     print(name)
    if name in new_attack:
        new_attack[name] += Attack_sign[ele][0]
    else:
        new_attack[name] = Attack_sign[ele][0]
print(sorted(new_attack.items(), key = lambda i:i[1],reverse = -1) )